In [1]:
import mask_dataset
from config import config
from model import MaskCNN

import os
import random
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [2]:
random.seed(config.random_seed)
torch.manual_seed(config.random_seed)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
dataset = mask_dataset.MaskDataset(root_dir='../dataset', img_dim=config.img_dim)

In [5]:
train_loader, valid_loader = dataset.get_data_loaders(batch_size=config.batch_size, use_shuffle=config.use_shuffle, split_ratio=config.split_ratio)

In [6]:
model = MaskCNN(len(dataset.labels), config.img_dim, base_filter_size=config.base_filter_size)
model = model.to(device)
model.eval()

MaskCNN(
  (classifier): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(256, 384, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(384, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 640, kernel_size=(3, 3), s

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)

In [ ]:
avg_loss = 0
best_vloss = 999999

for epoch in range(config.epochs):
    # traning
    model.train(True)
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        images, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        out_labels = model(images)
        loss = loss_fn(out_labels, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # print every config.log_interval mini-batches
        if i % config.log_interval == config.log_interval - 1:
            avg_loss = running_loss / config.log_interval
            print(f'[epoch_{epoch + 1}, batch_{i + 1}]\ttrain_loss: {avg_loss:.3f}')
            running_loss = 0.0
            
    # validation
    model.train(False)
    running_vloss = 0.0
    with torch.no_grad():
        for i, vdata in enumerate(valid_loader):
            vimages, vlabels = vdata[0].to(device), vdata[1].to(device)
            vout_labels = model(vimages)
            vloss = loss_fn(vout_labels, vlabels)
            running_vloss += vloss.item()

    avg_vloss = running_vloss / (i + 1)
    print(f'[epoch_{epoch + 1} ended]\ttrain_loss: {avg_loss:.3f}, valid_loss: {avg_vloss:.3f}\n')

    # save the best model.
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        torch.save(model.state_dict(), f'../trained/MaskCNN_0.pt')

print('\ndone.')

In [7]:
model.load_state_dict(torch.load('../trained/MaskCNN_0.pt'))
model.eval()

MaskCNN(
  (classifier): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(256, 384, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(384, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Conv2d(512, 640, kernel_size=(3, 3), s

In [8]:
total = 0
correct = 0

total_per_class = [0, 0, 0, 0, 0]
correct_per_class = [0, 0, 0, 0, 0]

with torch.no_grad():
    for data in valid_loader:
        images, labels = data[0].to(device), data[1].to(device)

        out_labels = model(images)
        preds = torch.argmax(out_labels.data, dim=1)
        total += labels.shape[0]
        correct += (preds == labels).sum().item()
        total_per_class[labels[0]] += labels.shape[0]
        correct_per_class[labels[0]] += (preds == labels).sum().item()

In [9]:
total = np.array(total)
correct = np.array(correct)
print(f'val - accuracy: {np.around(100 * correct / total, 2)}%')

total_per_class = np.array(total_per_class)
correct_per_class = np.array(correct_per_class)
acc_per_class = np.around(100 * correct_per_class / total_per_class, 2)

print('val - per class accuracy:')
for idx, acc in enumerate(acc_per_class):
    print(f' - {dataset.label_to_str[idx]}: {acc}%')

print(f'val - avg accuracy: {np.around(acc_per_class.mean(), 2)}%')

val - accuracy: 80.5%
val - per class accuracy:
 - no-mask: 96.88%
 - surgical: 81.82%
 - cloth: 74.22%
 - n95: 76.56%
 - n95-valve: 73.44%
val - avg accuracy: 80.58%


In [10]:
total = 0
correct = 0

total_per_class = [0, 0, 0, 0, 0]
correct_per_class = [0, 0, 0, 0, 0]

with torch.no_grad():
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)

        out_labels = model(images)
        preds = torch.argmax(out_labels.data, dim=1)
        total += labels.shape[0]
        correct += (preds == labels).sum().item()
        total_per_class[labels[0]] += labels.shape[0]
        correct_per_class[labels[0]] += (preds == labels).sum().item()

In [11]:
total = np.array(total)
correct = np.array(correct)
print(f'train - accuracy: {np.around(100 * correct / total, 2)}%')

total_per_class = np.array(total_per_class)
correct_per_class = np.array(correct_per_class)
acc_per_class = np.around(100 * correct_per_class / total_per_class, 2)

print('train - per class accuracy:')
for idx, acc in enumerate(acc_per_class):
    print(f' - {dataset.label_to_str[idx]}: {acc}%')

print(f'train - avg accuracy: {np.around(acc_per_class.mean(), 2)}%')

train - accuracy: 100.0%
train - per class accuracy:
 - no-mask: 100.0%
 - surgical: 100.0%
 - cloth: 100.0%
 - n95: 100.0%
 - n95-valve: 100.0%
train - avg accuracy: 100.0%
